In [1]:
# Last Revised - 2024.04.17
# Create flood products from HEC-RAS 2D output

import h5py
from shapely.geometry import Point, LineString, Polygon, MultiPolygon, box, mapping, shape
import geopandas as gpd
import pandas as pd
import numpy as np
import os

import rasterio
from rasterio.transform import from_origin
from scipy.interpolate import griddata
import numpy as np
from rasterio.transform import Affine

from rasterio.enums import Resampling
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject
from rasterio.features import shapes

import rasterio.mask

from osgeo import gdal, gdalconst

In [2]:
# Path to the HDF file
hdf_file_path = r'E:\HECRAS_2D_12070205\base_model_20240414_copy\BLE_LBSG_501.p02.hdf'
str_dem_path = r'E:\HECRAS_2D_12070205\BLE_12070205_Engineering_Models\Engineering Models\Hydraulic Models\RAS_Submittal\LBSG_501\Input\Terrain\Terrain4.DEM_3.tif'

# Specify the path where you want to save the GeoPackage file
input_gpkg = r'E:\sample_2d_output\sample_flooded_cells.gpkg'

str_dem_output_folder = r'E:\working\large_dem'

In [3]:
# Read the 
# Read the GeoPackage file
gdf_cells_wsel = gpd.read_file(input_gpkg, layer='02_cells_wsel')

# Get unique values of 'cell_idx' and sort them
list_unique_indices_sorted = sorted(gdf_cells_wsel['cell_idx'].unique())

In [4]:
# ------------------------
def fn_get_group_names(hdf5_file_path, group_path):
    """
    Retrieve the names of groups within a specified HDF5 file under a given group path.

    Parameters:
    hdf5_file_path (str): The file path to the HDF5 file.
    group_path (str): The path to the group whose subgroups' names are to be retrieved.

    Returns:
    list or None: A list containing the names of groups found under the specified group path. 
                  Returns None if the group path does not exist in the HDF5 file.
    """
    try:
        with h5py.File(hdf5_file_path, 'r') as hdf_file:
            # Check if the specified group path exists
            if group_path in hdf_file:
                group = hdf_file[group_path]

                # Extract names of HDF5 Group objects
                group_names = [name for name in group if isinstance(group[name], h5py.Group)]

                return group_names
            else:
                print(f"Group '{group_path}' not found in the HDF5 file.")
                return None
    except Exception as e:
        print(f"An error occurred: {e}")
# ------------------------

In [5]:
# Specify the HDF5 file path and group path
group_path = '/Geometry/2D Flow Areas/'

# Get names of HDF5 Group objects in the specified group
list_group_names = fn_get_group_names(hdf_file_path, group_path)

# for now... lets assume that there is only one 2D area
str_cell_info_path = '/Geometry/2D Flow Areas/Cell Info'

str_cell_center_point_path = group_path + list_group_names[0] + '/' + 'Cells Center Coordinate'

# From the plan HDF, get the cells center coordinates (not the centroid)
with h5py.File(hdf_file_path, 'r') as hdf_file:
    arr_cell_center_coords = hdf_file[str_cell_center_point_path][:]
    
# Filter the array to those cells that are wet
arr_center_coords_wet_cells = arr_cell_center_coords[list_unique_indices_sorted]

geometry = [Point(x, y) for x, y in arr_center_coords_wet_cells]

# Create a GeoDataFrame
gdf_center_points = gpd.GeoDataFrame(geometry=geometry, columns=['geometry'], crs=gdf_cells_wsel.crs)

# create a new coloumn that contains the cell index
gdf_center_points['cell_idx'] = list_unique_indices_sorted

# from geodataframe, create dataframe from only the 'cell_idx' and 'wsel' coloumns
df_wsel = gdf_cells_wsel[['cell_idx', 'wsel']]

# left join the WSEL per cell to pobulate the geodataframe of cells with WSEL
gdf_wsel_wet_cells = pd.merge(gdf_center_points, df_wsel, on='cell_idx', how='left')

# set constant arbitraty value used to disolve geometry
gdf_cells_wsel['val'] = 1

# create a geodataframe of the merged cells
gdf_dissolved = gdf_cells_wsel.dissolve(by='val')

# get shapely geometry of the first merged polygon
shp_wet_cells = gdf_dissolved.geometry.iloc[0]

# Convert Shapely polygon to GeoJSON-like format
geom = mapping(shp_wet_cells)

In [6]:
# Get the resolution of the ground DEM
# Open the raster file
dataset = gdal.Open(str_dem_path)

# Get raster resolution
flt_pixel_width = dataset.GetGeoTransform()[1]

# Close the dataset raster file
dataset = None  # Release the reference to the dataset

In [7]:
#gdf_wsel_wet_cells

In [8]:
# From the points in gdf_wsel_wet_cells wrap a convex hull
# Check each cell's polygon (gdf_cells_wsel) faces.  If the center of that cells face is not within the convex hull
# then create a point at the center of that face.  If there are duplicate points on the same spot, average the
# wsel value of these spots and drop

# TODO -2 024.04.16 -  maybe this should be vertex and note center point

from scipy.spatial import ConvexHull
from shapely.geometry import Point, Polygon
import geopandas as gpd

# -------------
def fn_compute_convex_hull(points):
    hull = ConvexHull(points)
    return [points[vertex] for vertex in hull.vertices]

def fn_calculate_vertex_points(polygon):
    list_points = []
    coords = polygon.exterior.coords
    num_coords = len(coords)
    for i in range(num_coords):  # Iterate over vertex
        x, y = coords[i]
        list_points.append(Point(x, y))
    return list_points

def fn_is_point_inside_convex_hull(point, hull_points):
    hull_polygon = Polygon(hull_points)
    return hull_polygon.contains(point)

# Define a function to merge points within a certain distance threshold
def merge_points_within_threshold(gdf, threshold):
    merged_points = []
    grouped = gdf.groupby(gdf.geometry.apply(lambda x: (round(x.x, 2), round(x.y, 2))))
    
    for _, group in grouped:
        if len(group) > 1:
            merged_point = Point(group.geometry.x.mean(), group.geometry.y.mean())
            avg_wsel = group['wsel'].mean()  # Calculate the average of 'wsel' column
            merged_points.append({'geometry': merged_point, 'wsel': avg_wsel})
        else:
            merged_points.append({'geometry': group.iloc[0].geometry, 'wsel': group.iloc[0]['wsel']})
    
    return gpd.GeoDataFrame(merged_points)
# -------------

# List to store created points
created_points = []
list_cell_wsel = []

# Wrap Convex Hull
convex_hull_points = fn_compute_convex_hull(gdf_wsel_wet_cells.geometry.apply(lambda point: (point.x, point.y)).tolist())

# Iterate through cell polygons and perform checks
for _, cell_polygon in gdf_cells_wsel.iterrows():
    vertex_points = fn_calculate_vertex_points(cell_polygon.geometry)
    for v_point in vertex_points:
        if not fn_is_point_inside_convex_hull(v_point, convex_hull_points):
            created_points.append(v_point)
            list_cell_wsel.append(cell_polygon['wsel'])

# Create a new GeoDataFrame from created points
gdf_additional_points = gpd.GeoDataFrame(geometry=created_points)

gdf_additional_points['wsel'] = list_cell_wsel
gdf_additional_points.crs = gdf_wsel_wet_cells.crs

# Remove the duplicate points... that sit on two cells
# Set the distance threshold
threshold_distance = 0.01  # in feet

# Merge points within the threshold distance
gdf_additional_points_duplicates_removed = merge_points_within_threshold(gdf_additional_points, threshold_distance)

gdf_additional_points_duplicates_removed.crs = gdf_wsel_wet_cells.crs


points1 = gdf_wsel_wet_cells[['geometry', 'wsel']].copy()

if len(gdf_additional_points_duplicates_removed) > 0:
    points2 = gdf_additional_points_duplicates_removed[['geometry', 'wsel']].copy()
    points = pd.concat([points1, points2], axis=0)
else:
    points = points1

In [9]:
# create the points of computed wsel
x = points.geometry.x.values
y = points.geometry.y.values
z = points['wsel'].values

# Define the extent of the cell point raster
xmin, ymin, xmax, ymax = gdf_dissolved.total_bounds
x_res = flt_pixel_width  # resolution in x direction
y_res = flt_pixel_width  # resolution in y direction

# Create the grid
xi = np.arange(xmin, xmax, x_res)
yi = np.arange(ymin, ymax, y_res)
xi, yi = np.meshgrid(xi, yi)

# Perform bilinear interpolation
zi = griddata((x, y), z, (xi, yi), method='linear')

# Define the affine transformation
transform = Affine.translation(xmin, ymin) * Affine.scale(x_res, y_res)

In [ ]:
# ~~~~~~~~~~~~~~~~~~~~~~~~
# Begin the creation of the raster products
list_raster_to_purge = []

# Define output raster file path
output_raster_path_01 = os.path.join(str_dem_output_folder,'01_wsel_interp.tif')

# Write the interpolated raster to a GeoTIFF file
with rasterio.open(output_raster_path_01, 'w', driver='GTiff', height=zi.shape[0], width=zi.shape[1], count=1,
                   dtype=zi.dtype,
                   crs=gdf_wsel_wet_cells.crs,
                   transform=transform) as dst:
    dst.write(zi, 1)
    
list_raster_to_purge.append(output_raster_path_01)

In [10]:
# Convert Shapely polygon to GeoJSON-like format
geom = mapping(shp_wet_cells)

# Open the raster dataset in read mode
with rasterio.open(output_raster_path_01, 'r') as dst:

    # Mask the interpolated raster with the polygon
    zi_masked, transform_masked = rasterio.mask.mask(dst, [geom], crop=True, nodata=np.nan)  # Specify nodata value as np.nan
    

In [11]:
# Define output masked raster file path
# Define output raster file path
output_masked_raster_path_02 = os.path.join(str_dem_output_folder,'02_wsel_interp_masked.tif')

# Write the masked interpolated raster to a GeoTIFF file
with rasterio.open(output_masked_raster_path_02, 'w', driver='GTiff', height=zi_masked.shape[1], width=zi_masked.shape[2], count=1,
                   dtype=zi_masked.dtype,
                   crs=gdf_wsel_wet_cells.crs,
                   transform=transform_masked,
                   nodata=np.nan) as dst_masked:  # Specify nodata value as np.nan
    dst_masked.write(zi_masked[0], 1)
    
list_raster_to_purge.append(output_masked_raster_path_02)

In [12]:
try:
    # Open and read the first raster
    with rasterio.open(output_masked_raster_path_02) as src1:
        raster1 = src1.read(1)
        transform1 = src1.transform
        crs1 = src1.crs
        
        try:
            # Open and read the second raster (ground dem raster)
            with rasterio.open(str_dem_path) as src2:
                raster2 = src2.read(1)
                transform2 = src2.transform
                crs2 = src2.crs

                # Get common extent
                bbox = gpd.GeoDataFrame(geometry=[box(*src1.bounds), box(*src2.bounds)], crs=crs1)
                common_extent = bbox.unary_union.bounds

                # Resample and align rasters
                dst_crs = crs1
                transform, width, height = calculate_default_transform(crs2, dst_crs, src2.width, src2.height, *src2.bounds)
                kwargs = src2.meta.copy()
                kwargs.update({
                    'crs': dst_crs,
                    'transform': transform,
                    'width': width,
                    'height': height
                })

                raster2_aligned = np.zeros_like(raster1)
                reproject(
                    source=raster2,
                    destination=raster2_aligned,
                    src_transform=src2.transform,
                    src_crs=src2.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)

                # Clip rasters
                output_raster_path_03 = os.path.join(str_dem_output_folder,'03_wsel_interp_mask_align.tif')
                output_raster_path_04 = os.path.join(str_dem_output_folder,'04_ground_align.tif')
                
                with rasterio.open(output_raster_path_03, 'w', **src1.meta) as dst1:
                    out_img1, out_transform1 = mask(src1, [bbox.iloc[0]['geometry']], crop=True)
                    dst1.write(out_img1)

                with rasterio.open(output_raster_path_04, 'w', **src2.meta) as dst2:
                    out_img2, out_transform2 = mask(src2, [bbox.iloc[1]['geometry']], crop=True)
                    dst2.write(out_img2)
                    
                list_raster_to_purge.append(output_raster_path_03)
                list_raster_to_purge.append(output_raster_path_04)

        except rasterio.errors.RasterioIOError as e:
            print("Error opening or processing raster file:", e)
        
except rasterio.errors.RasterioIOError as e:
    print("Error opening raster file:", e)

In [13]:
# -------------------
def fn_raster_subtraction(raster1_path, raster2_path, output_folder):
    
    list_created_rasters = []
    
    # Open the first raster
    raster1 = gdal.Open(raster1_path, gdalconst.GA_ReadOnly)
    if raster1 is None:
        raise FileNotFoundError(f"Raster not found at {raster1_path}")

    # Open the second raster
    raster2 = gdal.Open(raster2_path, gdalconst.GA_ReadOnly)
    if raster2 is None:
        raise FileNotFoundError(f"Raster not found at {raster2_path}")

    # Get the extent and resolution of the first raster
    x_min, pixel_width, _, y_max, _, pixel_height = raster1.GetGeoTransform()
    x_max = x_min + raster1.RasterXSize * pixel_width
    y_min = y_max + raster1.RasterYSize * pixel_height

    # Resample the second raster to match the extent and resolution of the first raster
    #resampled_raster2_path = r'E:\working\05_ground_resampled_align.tif'
    resampled_raster2_path = os.path.join(output_folder,'05_ground_resampled_align.tif')
    
    gdal.Warp(resampled_raster2_path, raster2, format='GTiff', outputBounds=[x_min, y_min, x_max, y_max], xRes=pixel_width, yRes=pixel_height)

    # Perform raster subtraction
    raster1_data = raster1.ReadAsArray()
    resampled_raster2 = gdal.Open(resampled_raster2_path, gdalconst.GA_ReadOnly)
    resampled_raster2_data = resampled_raster2.ReadAsArray()

    result_data = raster1_data - resampled_raster2_data

    # Create output raster
    driver = gdal.GetDriverByName('GTiff')
    
    output_raster_path_06 = os.path.join(output_folder,'06_depth.tif')
    output_raster = driver.Create(output_raster_path_06, raster1.RasterXSize, raster1.RasterYSize, 1, gdalconst.GDT_Float32)

    # Set projection and transform
    output_raster.SetProjection(raster1.GetProjection())
    output_raster.SetGeoTransform(raster1.GetGeoTransform())

    # TODO - 2024.02.23 -- likely writing two raster bands
    # Write result data to output raster
    output_band = output_raster.GetRasterBand(1)
    output_band.WriteArray(result_data)

    # Close rasters
    output_band = None
    output_raster = None
    raster1 = None
    resampled_raster2 = None
    
    list_created_rasters.append(output_raster_path_06)
    
    return(list_created_rasters)
# -------------------

# Perform raster subtraction
list_created_rasters = fn_raster_subtraction(output_raster_path_03, output_raster_path_04, str_dem_output_folder)

# Append the items in list_created_raster to list_raster_to_purge

In [14]:
# Build the raster file paths
output_raster_path_06 = os.path.join(str_dem_output_folder,'06_depth.tif')
output_raster_path_07 = os.path.join(str_dem_output_folder,'07_depth_with_nan.tif')

with rasterio.open(output_raster_path_06, 'r') as src:
    # Read raster data
    data = src.read(1)

    # Set values less than zero to NaN
    data[data < 0] = float('nan')

    # Copy metadata
    kwargs = src.meta.copy()

    # Update metadata for the new file
    kwargs.update(dtype=rasterio.float32)

    # Write the modified data to a new GeoTIFF file
    with rasterio.open(output_raster_path_07, 'w', **kwargs) as dst:
        dst.write(data, 1)

In [15]:
# -------------------
def fn_raster_addition_wsel(ground_raster_path, depth_raster_path, output_folder):

    # Open the GeoTIFF rasters
    ground_raster = gdal.Open(ground_raster_path)
    depth_raster = gdal.Open(depth_raster_path)

    # Read raster data as NumPy arrays
    ground_data = ground_raster.ReadAsArray()
    depth_data = depth_raster.ReadAsArray()

    # Convert depth data to float, as NaN is a floating-point concept
    depth_data = depth_data.astype(float)

    # Mask NaN values in depth data
    depth_data[np.isnan(depth_data)] = np.nan

    # Add the two rasters where depth is not NaN
    result = np.where(~np.isnan(depth_data), ground_data + depth_data, np.nan)

    # Save the result to a new GeoTIFF
    
    output_path = os.path.join(output_folder, '08_wsel_with_nan.tif')
    
    driver = gdal.GetDriverByName("GTiff")
    output_raster = driver.Create(output_path, ground_raster.RasterXSize, ground_raster.RasterYSize, 1, gdal.GDT_Float32)
    output_raster.SetProjection(ground_raster.GetProjection())
    output_raster.SetGeoTransform(ground_raster.GetGeoTransform())
    output_band = output_raster.GetRasterBand(1)
    output_band.WriteArray(result)
    output_band.FlushCache()

    # Close the rasters
    ground_raster = None
    depth_raster = None
    output_raster = None
# -------------------

In [16]:
output_raster_path_05 = os.path.join(str_dem_output_folder,'05_ground_resampled_align.tif')

fn_raster_addition_wsel(output_raster_path_05, output_raster_path_07, str_dem_output_folder)

In [17]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~
def fn_make_floodplain_gdf(str_input_raster_filepath, flt_minvalue, flt_max_value):

    """
    Converts a raster file representing flooded areas to a GeoDataFrame of a single MultiPolygon representing the floodplain.

    Parameters:
    - str_input_raster_filepath (str): File path of the input raster file.
    - flt_downscale_size (float): Resolution to which the raster should be downscaled.
    - flt_minvalue (float): minimum value of depth allowed
    - flt_max_value (float): maximum value of depth allowed

    Returns:
    - gdf_single_floodplain (geopandas.GeoDataFrame): GeoDataFrame containing a merged MultiPolygon representing the floodplain.
    """
    
    # ----------------------
    # open the raster file and convert to 'binary' of flooded=1 and null = 255
    with rasterio.open(str_input_raster_filepath) as src:
        # Read the raster data as a numpy array
        data = src.read(1)

        # Set values greater than flt_minvalue and <= flt_max_value to 1, others to 255
        data = np.where((data > flt_minvalue) & (data <= flt_max_value), 1, 255).astype('uint8')


        # Create a new raster file with the updated data type and nodata value
        profile = src.profile
        profile.update(dtype=rasterio.uint8, nodata=255)

        # Use in-memory storage
        output_raster_memory = rasterio.MemoryFile()
        with output_raster_memory.open(**profile) as dst:
            dst.write(data, 1)

        #print("Conversion completed. Output raster stored in memory.")

    # ----------------------
    # convert the downscaled binary raster to geodataframe of polygons
    # Open the input GeoTIFF file
    with output_raster_memory.open() as src:
        # Read the raster data
        data = src.read(1, masked=True)  # using masked=True to handle NoData as a mask

        # Set all non-null values to 1
        data = np.where(data.mask, 255, 1).astype('uint8')

        # Extract shapes from the raster data
        shapes_gen = shapes(data, mask=data != 255, transform=src.transform)

        # Convert shapes to Shapely geometries and create a GeoDataFrame
        geometries = [shape(geom) for geom, _ in shapes_gen]
        gdf = gpd.GeoDataFrame(geometry=geometries, crs=src.crs)

    # Merge all polygons into a single MultiPolygon
    merged_polygon = gdf['geometry'].unary_union

    # Create a new GeoDataFrame with the merged MultiPolygon
    gdf_single_floodplain = gpd.GeoDataFrame(geometry=[merged_polygon], crs=gdf.crs)

    # Close downscaled_raster_memory to release the memory
    output_raster_memory.close()

    return(gdf_single_floodplain)
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [18]:
flt_minvalue = 0
flt_max_value = 10000

gdf_floodplain = fn_make_floodplain_gdf(output_raster_path_07, flt_minvalue, flt_max_value)

# Specify the file path where you want to save the shapefile
output_shapefile = os.path.join(str_dem_output_folder,'09_floodplain_ar.gpkg')

# Save the GeoDataFrame as a shapefile
gdf_floodplain.to_file(output_shapefile, driver="GPKG")

In [19]:
# Temp - create a point geopackage of the points for wsel interpolation
output_path = os.path.join(str_dem_output_folder,'10_cell_pnts.gpkg')

gdf_wsel_wet_cells.to_file(output_path, layer='01_hecras_cells_pnt', driver="GPKG")
gdf_additional_points.to_file(output_path, layer='02_additional_vertex_pnt', driver="GPKG")

In [20]:
for item in list_created_rasters:
    list_raster_to_purge.append(item)

In [21]:
# Probably want to keep 5(ground), 7(depth) & 8(wsel)

for file_path in list_raster_to_purge:
    try:
        os.remove(file_path)
        print(f"File '{file_path}' deleted successfully.")
    except OSError as e:
        print(f"Error deleting file '{file_path}': {e}")

File 'E:\working\large_dem\01_wsel_interp.tif' deleted successfully.
File 'E:\working\large_dem\02_wsel_interp_masked.tif' deleted successfully.
File 'E:\working\large_dem\03_wsel_interp_mask_align.tif' deleted successfully.
File 'E:\working\large_dem\04_ground_align.tif' deleted successfully.
File 'E:\working\large_dem\06_depth.tif' deleted successfully.
